<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description-Task-1:-RNN-Language-Modelling-(30-+10-Points)" data-toc-modified-id="Description-Task-1:-RNN-Language-Modelling-(30-+10-Points)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description Task 1: RNN Language Modelling (30 +10 Points)</a></span><ul class="toc-item"><li><span><a href="#1a)-Language-Modelling-(30-Points)" data-toc-modified-id="1a)-Language-Modelling-(30-Points)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1a) Language Modelling (30 Points)</a></span></li><li><span><a href="#Conditional-Generation-(10-Points)" data-toc-modified-id="Conditional-Generation-(10-Points)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Conditional Generation (10 Points)</a></span></li></ul></li><li><span><a href="#Code-for-Task-1" data-toc-modified-id="Code-for-Task-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code for Task 1</a></span><ul class="toc-item"><li><span><a href="#Task-1.1" data-toc-modified-id="Task-1.1-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Task 1.1</a></span><ul class="toc-item"><li><span><a href="#Setup-and-preparation" data-toc-modified-id="Setup-and-preparation-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Setup and preparation</a></span></li><li><span><a href="#Data-preprocessing" data-toc-modified-id="Data-preprocessing-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Data preprocessing</a></span></li><li><span><a href="#RNN-construction" data-toc-modified-id="RNN-construction-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>RNN construction</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#Sample-and-test" data-toc-modified-id="Sample-and-test-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Sample and test</a></span></li><li><span><a href="#Output" data-toc-modified-id="Output-2.1.6"><span class="toc-item-num">2.1.6&nbsp;&nbsp;</span>Output</a></span></li></ul></li></ul></li></ul></div>

# Natural Language Understanding: Project 1

[__Natural Language Understanding, Spring 2018, ETHZ__](http://www.da.inf.ethz.ch/teaching/2018/NLU/)

[__Project 1__ (ETHZ network)](http://www.da.inf.ethz.ch/teaching/2018/NLU/material/project.pdf)

# Project to-do list:

Somewhat in order of importance:

- change code to unroll RNN in time instead of using dynamic_rnn
- make sure the target data fed into the crossentropy metric is really in correct form
- try own implementation of basic RNN cell instead of TF-prefab RNN or LSTM cell
- (Done 2.4.2018): change implementation to use the Xavier initializer instead of the uniform distribution currently used (see below)
- (Done 2.4.2018): change all `tf.Variable` variable inits to the better practice form like `W = tf.get_variable(name='example', shape=[784, 256], initializer=tf.contrib.layers.xavier_initializer())` which also includes the proper weight init
- include dropout at input and/or RNN cell level for regularization
- clean up namespaces, tensor naming
- (Started 2.4.2018, but TBC) build in all reporting for Tensorboard
- implement perplexity function
- adapt code to allow for differently sized timesteps
- make arrangements to save trained model
- adapt code to allow for use of pretrained word2vec embedding
- implement result output function
- implement sampling function for conditional text generation

## Description Task 1: RNN Language Modelling (30 +10 Points)

### 1a) Language Modelling (30 Points)
Your task is to build a simple LSTM language model. To be precise, we assume that words are independent given the recurrent hidden state; we compute a new hidden state given the last hidden state and last word, and predict the next word given the hidden state:
$$ P(w_1,\dots,w_n) = 􏰀\prod_{t=1}^{n}P(w_t|\mathbf{h}_t)$$
$$ P(w_t|\mathbf{h}_t) = \text{softmax}(\mathbf{Wh}_t)$$
$$ \mathbf{h}_t = f(\mathbf{h}_{t−1}, w_{t-1}^{*})$$

where $f$ is the LSTM recurrent function, $\mathbf{W} \in \mathbb{R}^{|V|×d}$ are softmax weights and $\mathbf{h_0}$ is either an all-zero constant or a trainable parameter.
You can use the tensorflow cell implementation __[1](https://www.tensorflow.org/api_docs/python/tf/nn/rnn_cell)__ to carry out the recurrent computation in $f$. However, you must construct the actual RNN yourself (e.g. don’t use tensorflow’s `static_rnn` or `dynamic_rnn` or any other RNN library). That means, you will need to use a python loop that sets up the unrolled graph. To make your life simpler, please follow these design choices:

__Model and Data specification__

- Use a special sentence-beginning symbol `<bos>` and a sentence-end symbol `<eos>` (please use exactly these, including brackets). The `<bos>` symbol is the input, when predicting the first word and the `<eos>` symbol you require your model to predict at the end of every sentence.
- Use a maximum sentence length of 30 (including the `<bos>` and `<eos>` symbol). Ignore longer sentences during training and testing.
- Use a special padding symbol `<pad>` (please use exactly this, including brackets) to fill up sentences of length shorter than 30. This way, all your input will have the same size.
- Use a vocabulary consisting of the 20K most frequent words in the training set, including the symbols `<bos>`, `<eos>`, `<pad>` and `<unk>`. Replace out-of-vocabulary words with the `<unk>` symbol before feeding them into your network (don’t change the file content).
- Provide the ground truth last word as input to the RNN, not the last word you predicted. This is common practice.
- Language models are usually trained to minimize the cross-entropy. Use tensorflow’s `tf.nn.sparse_softmax_cross_entropy_with_logits` to compute the loss (*This operation fuses the computation of the soft-max and the cross entropy loss given the logits. For numerical stability, it’s very important to use this function.*). Use the AdamOptimizer with default parameters to minimize the loss. Use `tf.clip_by_global_norm` to clip the norm of the gradients to 5.
- Use a batch size of 64.
- Use the data at __[6](https://polybox.ethz.ch/index.php/s/qUc2NvUh2eONfEB)__. Don’t pre-process the input further. All the data is already white-space tokenized and
lower-cased. One sentence per line.
- To initialize your weight matrices, use the `tf.contrib.layers.xavier_initializer()` initializer introduced in __[5](http://jmlr.org/proceedings/papers/v9/glorot10a/glorot10a.pdf)__.

__Experiments__
All experiments should not run for longer than, say, four hours on the GPU. For this task, your
grade won’t improve with performance.

- __Experiment A__: Train your model with word-embedding dimensionality of 100 and a hidden state size of 512 and compute sentence perplexity on the evaluation set (see submission format below).
- __Experiment B__: It is common practice, to pretrain word embeddings using e.g. `word2vec`. This should make your model train faster as words will come already with some useful representation. Use the code at __[3](http://da.inf.ethz.ch/teaching/2018/NLU/material/load_embeddings.py)__ to load these word embeddings __[4](https://polybox.ethz.ch/index.php/s/cpicEJeC2G4tq9U)__ trained on the same corpus. Train your model again and compute evaluation perplexity.
- __Experiment C__: It is often desirable to make the LSTM more powerful, by increasing the hidden dimensionality. However, this will naturally increase the parameters $\mathbf{W}$ of the softmax. As a compromise, one can use a larger hidden state, but down-project it before the softmax. Increase the hidden state dimensionality from 512 to 1024, but down-project $h_t$ to dimensionality 512 before predicting $w_t$ as in
$$ \mathbf{\tilde{h}}_t = \mathbf{W}_P\mathbf{h}_t$$
where $W_P$ are parameters. Train your model again and compute evaluation perplexity.

__Submission and grading__
- Grading scheme: 100% correctness.
- Deadline April 20th, 23:59:59.
- You are not allowed to copy-paste any larger code blocks from existing implementations.
- Hand in
    - Your python code
    - __Three__ result files containing sentence-level perplexity numbers on the __test__ set (to be distributed) for
all three experiments. Recall that perplexity of a sentence $S = ⟨w_1, \dots , w_n⟩$ with respect to your model $p(w_t|w_1, \dots, w_{t−1})$ is defined as
$$ \text{Perp} = 2^{-\frac{1}{n} \sum_{t=1}^{n}\log_2 p(w_t|w_1,\dots,w_{t−1})}$$
The `<eos>` symbol is part of the sequence, while the `<pad>` symbols (if any) are not. Be sure to have the basis of the exponential and the logarithm match.<br>
__Input format sentences.test__<br>
One sentence (none of them is longer than 28 tokens) per line:<br>
         ```beside her , jamie bounced on his seat .
         i looked and saw claire montgomery looking up at me .
         people might not know who alex was , but they knew to listen to him .```<br>
__Required output format groupXX.perplexityY__<br>
(where XX is your group __number__ and Y ∈ {A,B,C} is the experiment). One perplexity number per line:<br>
         $10.232$<br>
         $2.434$<br>
         $5.232$<br>
Make sure to have equally many lines in the output as there are in the input – otherwise your submission will be rejected automatically.
    - You have to submit at https://cmt3.research.microsoft.com/NLUETHZ2018

### Conditional Generation (10 Points)
Let’s use your trained language model from above to generate sentences. Given an initial sequence of words, your are asked to __greedily__ generate words until either your model decides to finish the sentence (it generated `<eos>`) or a given maximum length has been reached. Note, that this task does not involve any training. Please see the tensorflow documentation on how to save and restore your model from above.
There are several ways how to implement the generation. For example, you can define a graph that computes just one step of the RNN given the last input and the last state (both from a new placeholder).
$$ \text{state}_t, p_t = f(\text{state}_{t−1},w_{t−1}) $$
That means, for a prefix of size $m$ and a desired length of $n$, you run this graph $n$ times. The first $m + 1$ times you take the input form the prefix. For the rest of the sequence, you take the most likely2 word $w^{t−1} = \text{argmax}_w p_{t−1}(w)$ from the last step.

- Grading scheme: 100% correctness.
- Deadline April 20th, 23:59:59.
- You are not allowed to copy-paste any larger code blocks from existing implementations.
- Hand in
    - Your python code
    - Your continued sentences of length up to 20. Use your trained model from experiment __C__ in task 1.1.
    __Input format sentences.continuation__ One sentence (of length less than 20) per line:<br>
         ```beside her ,
         i
         people might not know```<br>
    The `<bos>` symbol is not explicitly in the file, but you should still use it as the first input.<br>
    __Required output format groupXX.continuation__ (where XX is your __group number__)<br>
         ```beside her , something happened ! <eos>
         i do n’t recall making a noise , but i must have , because bob just looked up from his
         people might not know the answer . <eos>```
    - You have to submit at https://cmt3.research.microsoft.com/NLUETHZ2018


__Infrastructure__

You must use Tensorflow, but any programming language is allowed. However, we strongly recommend `python3`. You have access to two compute resources: Unlimited CPU usage on Euler and GPU usage on Leonhard. Note that the difference in speed is typically a factor between 10 and 100.

## Code for Task 1
### Task 1.1

#### Setup and preparation

Make sure you have done the following:

- Download data from https://polybox.ethz.ch/index.php/s/qUc2NvUh2eONfEB and unpack into `./data/` subdirectory
- Download embeddings from https://polybox.ethz.ch/index.php/s/cpicEJeC2G4tq9U and unpack into `./data/` subdirectory
- Download helper function from http://da.inf.ethz.ch/teaching/2018/NLU/material/load_embeddings.py and put into `./helpers/` subdirectory

In [11]:
import time
#import tqdm


import tensorflow as tf
import numpy as np

from collections import Counter 
from helpers.load_embeddings import load_embedding

%matplotlib inline

In [12]:
# Read training and evalutation data from files

with open('./data/sentences.train', 'r') as f:
    train_data = f.read()
    
with open('./data/sentences.eval', 'r') as f:
    eval_data = f.read()

In [13]:
# Have a peek at the given raw data

print('Training data sample:\n', 20*'=')
print(train_data[:100], '\n', 80*'.')

print('\n Evaluation data sample:\n', 20*'=')
print(eval_data[:100], '\n', 80*'.')

Training data sample:
`` i 've never had any ice-cream for myself , my father never let me have any . ''
`` and even if sh 
 ................................................................................

 Evaluation data sample:
he took my face in his hands and held my face where he wanted it as he kissed me senseless .
`` on t 
 ................................................................................


#### Data preprocessing

In [14]:
# Get sentences from data
train_text = ''.join(train_data)
train_sentences = train_text.split('\n')
print('Sample sentences:\n', train_sentences[:5])

# Make text contiguous again and break into words for vocabulary
words = ' '.join(train_sentences).split()
print('Sample words:\n', words[:20])

# Make a word counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
print('Top frequency words:\n', vocab[:20])

# Clip word counter to defined length and append special symbol words
symbols = ['<bos>', '<eos>', '<pad>', '<unk>']
vocab_length = 20000 # restrict to 20k most frequent tokens
vocab = vocab[:vocab_length-len(symbols)] # kick out the last 4 to replace w/ symbols
for each in symbols:
    vocab.append(each)
    
# Make a vocabulary to convert words to integers
vocab_to_int = {word: i for i, word in enumerate(vocab, 0)} # consider start with 1 if problems

# Make a vocabulary to get words from integers at the end
int_to_vocab = dict(enumerate(vocab))


Sample sentences:
 ["`` i 've never had any ice-cream for myself , my father never let me have any . ''", "`` and even if she was , the gargoyle could hardly halt me from harming her . ''", 'pigafetta repeated the threat , which had no apparent effect .', "best to barricade oneself , no ? ''", "`` why would he lie ? ''"]
Sample words:
 ['``', 'i', "'ve", 'never', 'had', 'any', 'ice-cream', 'for', 'myself', ',', 'my', 'father', 'never', 'let', 'me', 'have', 'any', '.', "''", '``']
Top frequency words:
 ['.', ',', 'the', 'i', 'to', 'and', '``', "''", 'a', 'he', 'of', 'you', 'was', 'her', 'it', 'she', 'in', 'his', 'that', '?']
3


In [16]:
# Encode training sentences to integers and insert symbol words where necessary

### ATTN: Remove next line after finishing, keeping data set small for speedup
train_sentences = train_sentences[:1000]

max_sentence_length = 30 # Given by task description
sentences_ints = []
for each in train_sentences:
    sentence = each.split()
    if len(sentence) < max_sentence_length-2: # -2 to allow for <bos>, <eos>
        sentence_int = [vocab_to_int['<bos>']] 
        sentence_int += [vocab_to_int[word] if word in vocab_to_int\
                             else vocab_to_int['<unk>'] for word in sentence]
        sentence_int.append(vocab_to_int['<eos>'])
        while len(sentence_int) < max_sentence_length:
            sentence_int.append(vocab_to_int['<pad>'])
        sentences_ints.append(sentence_int)
source = np.array(sentences_ints)

In [17]:
# Build batch generator for training    
def get_batches(source_arr, batch_size):
    '''Generator that returns features x and targets y of size:
            batch_size(number of sequences from source_arr)
                    x
            n_steps (length of sequence from source_arr)
    
    Inputs
    ------
    source_arr: A np.ndarray to generate features and targets from
    batch_size: An int number of sequences required per batch
    
    Outputs
    -------
    x: A np.ndarray of feature sequences according to parameters above
    y: A np.ndarray of target sequences according to paramteres above.
    '''
    # Determine number of sequence steps 
    n_steps = source_arr.shape[1] # = 30 in this task as we've padded to that length

    # Determine number of full batches that can be served
    n_batches = source_arr.shape[0] // batch_size
    
    # Crop source to make full batches only
    source_arr = source_arr[:n_batches * batch_size]
    
    for i in range(0, source_arr.shape[0], batch_size):
        # Feature sequence
        x = source_arr[i:i+batch_size, :]
        # Target sequence: Shift feature sequence by one step, wrap around
        y = np.zeros_like(x)
        y[:, :-1]= x[:, 1:] # Center part with shift
        y[:, -1] = x[:, 0] # Wrap around
        yield x, y

#### RNN construction

In [28]:
def build_input_layer(batch_size, num_steps=30):
    '''Build placeholders for inputs, targets, dropout factor
    
        TBC
    '''
    # Placeholders for inputs, targets and dropout coefficient
    with tf.variable_scope("input_layer") as scope:
        inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
        targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [41]:
def build_embedding_layer(vocab_length, embedding_size, inputs):
    '''Build embedding layer as one-hot encoding not viable
    
        TBC
    '''
    embedding_matrix = tf.get_variable(name = 'embedding_matrix', shape = [vocab_length, embedding_size], initializer = tf.random_uniform_initializer(-1, 1))
    embeddings = tf.nn.embedding_lookup(embedding_matrix, inputs, name='embedding_lookup')
    return embeddings

In [20]:
def build_lstm_layer(lstm_size, lstm_layers, batch_size, keep_prob):
    '''Build LSTM cells
    
        TBC
    '''
     
    # LSTM cell from library as in task description
    cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    
    # Fancier cell option with multiple possible layers
    # cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.BasicLSTMCell(lstm_size), output_keep_prob=keep_prob) for _ in range(lstm_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

In [51]:
def build_output_layer(lstm_layer_output, lstm_size, out_size):
    '''Build softmax output layer and return its output plus logits
    
        TBC
    '''
    # Reshape lstm_layer_output: Get one row for e/a step (i.e. over axis 1)
    rows_output = tf.concat(lstm_layer_output, 1)
    # Reshape again to get tensor with in_size (=size of lstm_layer_output) cols
    x = tf.reshape(rows_output, [-1, lstm_size])
    
    # Hook up LSTM outputs to softmax layer
    with tf.variable_scope('softmax') as scope:
        # Weight and bias variables: TBC: Project wants some funny initializer here (I think)
        xavier_initializer = tf.contrib.layers.xavier_initializer()
        softmax_w = tf.get_variable(name = "softmax_weight", shape = [lstm_size, out_size], initializer = xavier_initializer)
        softmax_b = tf.get_variable(name = "softmax_bias", shape = [out_size], initializer =tf.zeros_initializer)
    
    # Calculate logits from softmax layer
    logits = tf.add(tf.matmul(x, softmax_w), softmax_b)
    
    # Finally, get word probabilities from logits
    output = tf.nn.softmax(logits, name='predictions')
    
    return output, logits

In [22]:
def build_loss(logits, targets, lstm_size, out_size):
    ''' Calculate the loss from the logits and the targets.
    
            TBC
    '''
    # One-hot encode targets:
    # THIS IS FISHY, TBC!: We used embedding at the input, so I think
    # we should use the same again here for the reverse. However, I made a mess of 
    # the dimensions and used one-hot as a quick fix to check if the rest of the code
    # worked. 
    y_one_hot = tf.one_hot(targets, out_size)

    # Reshape encoded targets to fit logits
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [23]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training
    
        TBC
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [52]:
class LanguageModel:
    def __init__(self, out_size=vocab_length, batch_size=64, num_steps=30,
                   lstm_size=512, lstm_layers=1, learning_rate=0.001,
                   grad_clip=5, embedding_size=100, keep_prob=0.5):
        
        # Reset tensorflow graph for clean slate
        tf.reset_default_graph()
        
        
        # Build input layer
        self.inputs, self.targets, self.keep_prob = build_input_layer(batch_size,
                                                                      num_steps)

        # Build embedding layer
        embeddings = build_embedding_layer(vocab_length, embedding_size,
                                           self.inputs)

        # Build LSTM layer
        lstm, self.initial_state = build_lstm_layer(lstm_size, lstm_layers,
                                                    batch_size, self.keep_prob)

        # Unfold RNN: Replace 'dynamic_rnn' below with this once the rest works
        # state = initial_state
        # outputs = []
        # for time_step in range(num_steps):
        #     if time_step > 0:
        #         tf.get_variable_scope().reuse_variables()
        #     lstm_output, state = stacked_lstm(inputs, state)
        #     outputs.append(lstm_output)
        # state = final_state

        # Run data through RNN with dynamic_rnn, NOT FOR FINAL HAND-IN
        outputs, self.final_state = tf.nn.dynamic_rnn(lstm, embeddings,
                                                      initial_state=self.initial_state)

        # Build softmax output layer
        self.prediction, self.logits = build_output_layer(outputs, lstm_size, out_size)

        # Build loss
        self.loss = build_loss(self.logits, self.targets, lstm_size, out_size)

        # Build optimizer
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)
        

In [32]:
# Set hyperparameters

embedding_size = 100
lstm_size = 512
lstm_layers = 1
batch_size = 64
out_size = vocab_length
learning_rate = 0.001
keep_prob = 0.5
grad_clip = 5.0
num_steps = 30



#### Training

In [53]:
epochs = 10
save_every_n = 100
#saver = tf.train.Saver(max_to_keep=100)

model = LanguageModel()


# Summary / Variables used for Tensorboard
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = "train"
traindir = "{}/time-{}/".format(logdir, now)
loss_summary = tf.summary.scalar('loss', model.loss)
train_writer = tf.summary.FileWriter(traindir, tf.get_default_graph())



with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    
    counter = 0
    for ep in range(epochs):
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(source, batch_size):
            counter += 1
            start = time.time()
            feed_dict = {model.inputs: x,
                         model.targets: y,
                         model.keep_prob: keep_prob,
                         model.initial_state: new_state}
            batch_loss, new_state, loss_summ, _ = sess.run([model.loss,
                                                 model.final_state,
                                                 loss_summary,
                                                 model.optimizer],
                                                 feed_dict=feed_dict)
            stop = time.time()
            step = counter * batch_size
            train_writer.add_summary(loss_summ, step)
            
            print('Epoch: {}/{}\t'.format(ep+1, epochs),
                  'Training step: {}\t'.format(counter),
                  'Batch loss: {:.4f}\t'.format(batch_loss),
                  '{:4f} sec/batch'.format(stop-start))
    train_writer.close()
    #saver.save(sess, 'checkpoints/{}'.format(time.time()))

Epoch: 1/10	 Training step: 1	 Batch loss: 9.9246	 4.218139 sec/batch
Epoch: 1/10	 Training step: 2	 Batch loss: 9.6482	 4.405486 sec/batch
Epoch: 1/10	 Training step: 3	 Batch loss: 9.2186	 4.829539 sec/batch
Epoch: 1/10	 Training step: 4	 Batch loss: 8.5737	 4.868984 sec/batch
Epoch: 1/10	 Training step: 5	 Batch loss: 7.8536	 4.526288 sec/batch
Epoch: 1/10	 Training step: 6	 Batch loss: 7.2155	 4.397932 sec/batch
Epoch: 1/10	 Training step: 7	 Batch loss: 6.2262	 4.437476 sec/batch
Epoch: 1/10	 Training step: 8	 Batch loss: 5.4345	 4.547139 sec/batch
Epoch: 1/10	 Training step: 9	 Batch loss: 4.8648	 4.388584 sec/batch
Epoch: 1/10	 Training step: 10	 Batch loss: 4.0930	 5.998131 sec/batch
Epoch: 1/10	 Training step: 11	 Batch loss: 3.8794	 5.337300 sec/batch
Epoch: 1/10	 Training step: 12	 Batch loss: 3.4215	 4.625888 sec/batch
Epoch: 1/10	 Training step: 13	 Batch loss: 3.2233	 5.161476 sec/batch
Epoch: 1/10	 Training step: 14	 Batch loss: 3.4536	 4.820511 sec/batch
Epoch: 1/10	 Tr

Epoch: 8/10	 Training step: 117	 Batch loss: 2.4866	 3.343993 sec/batch
Epoch: 8/10	 Training step: 118	 Batch loss: 2.4224	 3.326502 sec/batch
Epoch: 8/10	 Training step: 119	 Batch loss: 2.6137	 3.354819 sec/batch
Epoch: 8/10	 Training step: 120	 Batch loss: 2.7522	 3.332219 sec/batch
Epoch: 9/10	 Training step: 121	 Batch loss: 2.4583	 3.358143 sec/batch
Epoch: 9/10	 Training step: 122	 Batch loss: 2.6633	 3.343000 sec/batch
Epoch: 9/10	 Training step: 123	 Batch loss: 2.4086	 3.380288 sec/batch
Epoch: 9/10	 Training step: 124	 Batch loss: 2.5425	 3.333640 sec/batch
Epoch: 9/10	 Training step: 125	 Batch loss: 2.7286	 3.387986 sec/batch
Epoch: 9/10	 Training step: 126	 Batch loss: 2.8592	 3.480861 sec/batch
Epoch: 9/10	 Training step: 127	 Batch loss: 2.4689	 3.383580 sec/batch
Epoch: 9/10	 Training step: 128	 Batch loss: 2.5122	 3.338818 sec/batch
Epoch: 9/10	 Training step: 129	 Batch loss: 2.6873	 3.475293 sec/batch
Epoch: 9/10	 Training step: 130	 Batch loss: 2.4790	 3.975580 se

In [54]:
tf.trainable_variables()

[<tf.Variable 'embedding_matrix:0' shape=(20000, 100) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/kernel:0' shape=(612, 2048) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/bias:0' shape=(2048,) dtype=float32_ref>,
 <tf.Variable 'softmax/softmax_weight:0' shape=(512, 20000) dtype=float32_ref>,
 <tf.Variable 'softmax/softmax_bias:0' shape=(20000,) dtype=float32_ref>]

#### Sample and test

#### Output